### Importing all the important libraries

In [1]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2
from glob import glob
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Defining the path for train and test images
data_dir_train = pathlib.Path(r"C:\Users\MANDAR\Downloads\upgrade\NN\Project\Skin cancer ISIC The International Skin Imaging Collaboration\Train")
data_dir_test = pathlib.Path(r"C:\Users\MANDAR\Downloads\upgrade\NN\Project\Skin cancer ISIC The International Skin Imaging Collaboration\Test")

In [3]:
image_count_train = len(list(data_dir_train.glob('*/*.jpg')))
print(image_count_train)
image_count_test = len(list(data_dir_test.glob('*/*.jpg')))
print(image_count_test)

2239
118


### Create a dataset

Define some parameters for the loader:

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir_train,
    validation_split= 0.4,  # You can adjust the validation split as needed
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size= batch_size  # You can adjust the batch size as needed
)

In [ ]:
val_ds =  tf.keras.preprocessing.image_dataset_from_directory(
    data_dir_test,
    validation_split=0.4,  # You can adjust the validation split as needed
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size= batch_size  # You can adjust the batch size as needed
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

### Visualize the data

In [ ]:
import matplotlib.pyplot as plt

import os
from PIL import Image
import matplotlib.pyplot as plt

def display_images_in_grid(images, grid_size=(1, 4)):
    fig, axs = plt.subplots(*grid_size, figsize=(15, 7))
    axs = axs.ravel()

    for i, img in enumerate(images):
        axs[i].imshow(img)
        axs[i].axis('off')

    plt.show()

def print_images_from_folders(root_dir, num_images=4):
    for foldername in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, foldername)

        # Check if the current item is a directory
        if os.path.isdir(folder_path):
            print(f"Images from folder: {foldername}")
            image_count = 0
            images_to_display = []

            # Loop through files in the folder
            for filename in os.listdir(folder_path):
                if image_count == num_images:
                    break

                file_path = os.path.join(folder_path, filename)

                # Check if the current item is a file and is an image (you can customize the list of valid extensions)
                if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
                    # Open the image and append it to the list
                    img = Image.open(file_path)
                    images_to_display.append(img)
                    image_count += 1

            # Display the images in a grid
            display_images_in_grid(images_to_display)

if __name__ == "__main__":
    print_images_from_folders(data_dir_train)




In [ ]:
UTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Create the model

In [ ]:
# Set the seed for reproducibility
seed = 123
tf.random.set_seed(seed)

# Define the image size
img_height, img_width = 180, 180
num_classes = 9

# Define the CNN model
model = models.Sequential()

# Preprocessing layer to rescale pixel values between (0, 1)
model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)))

# Convolutional and pooling layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten layer to transition from convolutional layers to dense layers
model.add(layers.Flatten())

# Dense layers
model.add(Flatten())
model.add(Dense(512,kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

### Compile the model

In [ ]:
your_loss_function_goes_here = 'sparse_categorical_crossentropy'

model.compile(optimizer="adam",
              loss=your_loss_function_goes_here,
              metrics=['accuracy'])

### Train the model

In [ ]:
epochs = 20
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=epochs,
                    batch_size=batch_size)

### Visualizing training results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Create the model, compile and train the model

In [ ]:
# Set the seed for reproducibility
seed = 123
tf.random.set_seed(seed)

# Define the image size
img_height, img_width = 180, 180
num_classes = 9

# Define the CNN model
model = models.Sequential()

# Preprocessing layer to rescale pixel values between (0, 1)
model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)))

# Convolutional and pooling layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten layer to transition from convolutional layers to dense layers
model.add(layers.Flatten())

# Dense layers
model.add(Flatten())
model.add(Dense(512,kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

### Compiling the model

In [ ]:
your_loss_function_goes_here = 'sparse_categorical_crossentropy'

model.compile(optimizer="adam",
              loss=your_loss_function_goes_here,
              metrics=['accuracy'])

### Training the model

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()